In [1]:
import sqlite3
import pandas as pd
import re
from pyarrow import feather

task = 'GUESSING'
id_list_files = [ 
    f"first_level/{task}-l1-list_{i}run.txt" 
    for i in range(1, 3) 
]
print(id_list_files)
df_list = []
for id_file in id_list_files:
    df_list.append(pd.read_table(id_file, sep=" ", header=None, names=["pid", "hcp_task", "fsf"]))
id_list = pd.concat(df_list, axis=0)
def parse_pid(pid):
    match = re.search(r"(?P<pid>HCD\d+)_(?P<session>V\d+)_MR", pid)

    if not match:
        return (None, None)

    pid = match.group("pid")
    session = match.group("session")

    return (match.group("pid"), match.group("session"))

id_list['pid'], id_list['session'] = zip(*id_list['pid'].apply(parse_pid))

# Connect to the SQLite database
conn = sqlite3.connect('hcpd_file_status.db')

task = "GUESSING"
# Query the database to load the entire 'files' table into a DataFrame
df = pd.read_sql_query(f"""
SELECT * FROM files WHERE 
file_type IS 'txt' AND
task IS '{task}' AND
data_type IS 'Parcellated'
""", conn)

# Close the connection
conn.close()
def has_missing(group):
    return 'missing' in group['status'].values
print(len(id_list[id_list.pid == 'HCD2562658']))
df_pid_missing_any = df.groupby(['pid', 'session']).apply(has_missing).reset_index(name='has_missing')
df_pid_missing_any = pd.merge(id_list, df_pid_missing_any, how='left', on=['pid', 'session'])
failed_to_extract_df = df_pid_missing_any[df_pid_missing_any['has_missing'].isna() | df_pid_missing_any['has_missing']]

['first_level/GUESSING-l1-list_1run.txt', 'first_level/GUESSING-l1-list_2run.txt']
3


In [2]:
df

,id,filepath,status,pid,session,task,data_type,file_type
0,1,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0007014...,built,HCD0007014,V1,GUESSING,Parcellated,txt
1,2,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD1127130...,built,HCD1127130,V1,GUESSING,Parcellated,txt
2,3,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0403224...,built,HCD0403224,V1,GUESSING,Parcellated,txt
3,4,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0718451...,built,HCD0718451,V1,GUESSING,Parcellated,txt
4,5,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD1849673...,built,HCD1849673,V1,GUESSING,Parcellated,txt
...,...,...,...,...,...,...,...,...
99003,99004,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0154431...,built,HCD0154431,V1,GUESSING,Parcellated,txt
99004,99005,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0154431...,built,HCD0154431,V1,GUESSING,Parcellated,txt
99005,99006,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0154431...,built,HCD0154431,V1,GUESSING,Parcellated,txt
99006,99007,/ncf/hcp/data/HCD-tfMRI-MultiRunFix/HCD0154431...,built,HCD0154431,V1,GUESSING,Parcellated,txt
